![title](yelp2.jpg)
# Smash the Snorkel Team Project: 
# A Weak Supervised Learning Study for Yelp Restaruant Review Classification 


## 1. Introduction

After manually labelling, our member realised that customers or visitors normally give a labelled "1" comments after they graded the business as a low grade (1 or 2 Star out of 5). Then, he did numerical study about the star and contribute couple of high coverage LFs to our Label Model.  

## 2. Loading Data

In [1]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
import pyspark
from pyspark.sql import SQLContext
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
import pyspark
from pyspark.sql import SQLContext
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
import pandas as pd
from pyspark.sql import SparkSession
from snorkel.labeling import PandasLFApplier,LFAnalysis
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()
pd_dev = pd.read_csv("review_dev_labelled.csv",header=0, index_col=0)
df_dev = spark.createDataFrame(pd_dev)

### 2.1 Data Source 1: Mongo DB

[To be continue:]

### 2.2 Data Source 2: Local JOSN Files

[To be continue:]

### 2.3 Data Filtering

In [ ]:
#First:  restaurant
biz_df = sqlContext.read.json("original_data/business.json") # Mongo Needed
biz_df.createOrReplaceTempView('biz_table')

[To be continue:]

### 2.4 Data Spliting

[To be continue:]

## 3. Labeling Functions

### 3.1 Manual Labeling

Our goal is to train a classifier over the Yelp data that could predict whether a comment/text is positive (containing suggestions or complaints) or negative. Hence, we have access to a large amount of unlabelled data in the form of Yelp comments with some metadata. After filtered the dataset, we split the filtered review data into training set, testing set and development set. Because originally the dataset doesn't contain any labels, our members manually labelled the reviews in development and testing sets (500 labels for each) with "1" for positive and "0" for negative. Therefore, the result of our label model can be tested in a reasonable way later.

### 3.2 Writing Labeling Functions 1: Text-based Functions

In this case, we used labelling functions (LFs) in Snorkel to apply the weak supervision approach by using programmatic rules and heuristics that assign labels to unlabelled training data. Individuals all have different methods to express their understanding and opinions, therefore, there is very unlikely exist completely same comments. Otherwise, if a single LF had high enough coverage to label our entire test dataset accurately, we don't need a classifier at all. In reality, most problems are not that simple. Instead, we usually need to combine multiple LFs to label our dataset, to improve the overall accuracy of the training labels we generate by factoring in multiple different signals, like keywords extraction, heuristics and numerical method.

In [1]:
#Shuo's Part:

<div class="alert alert-success">
<b> 3.2.1 Keywords Labeling Functions </b>

<p>For text/comment in the dataset, some Labeling Functions(LFs later in this paper) could be written based on just keywords lookups. 
    These will often follow a realtive similar execution pattern, so we can create a template and use the resources parameter to pass
    in specific keywords. Between, same for the labeling_function decorator, the LabelingFunction class wraps 
    a Python function (the f parameter), and we can use the resources parameter to pass in keyword arguments (here, our keywords to 
    lookup) to said function.
</div>


In [2]:
#Setup for Labeling function
import pyspark.sql.functions as F
from snorkel.labeling import LabelModel
from snorkel.labeling.apply.spark import SparkLFApplier
import pandas as pd
import numpy as np
#from snorkel.labeling import ,LFAnalysis
from snorkel.labeling import LFAnalysis
from pyspark.sql import Row
from snorkel.labeling.lf import labeling_function
#from snorkel.labeling.lf.nlp_spark import spark_nlp_labeling_function
from snorkel.preprocess import preprocessor

ABSTAIN = -1
NEGATIVE = 0
POSITIVE = 1

In [3]:
from snorkel.labeling import LabelingFunction

def keyword_lookup(x, keywords, label):
    if any(word in x.text.lower() for word in keywords):
        return label
    return ABSTAIN
def make_keyword_lf(keywords, label=0):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label)
    )
"""NEGATIVE comments talk about 'I love pizza', 'Love this resturant', etc."""
keyword_love = make_keyword_lf(keywords=["love"])

"""NEGATIVE comments talk about 'This food is perfect', 'Perfect people' to describe."""
keyword_perfect = make_keyword_lf(keywords=["perfect"])

"""POSITIVE comments talk about 'The waiter is rude', 'Their behaviour is rude', etc."""
keyword_rude = make_keyword_lf(keywords=["rude"])

"NEGATIVE comments talk about 'I love pizza', 'Love this resturant', etc."

"NEGATIVE comments talk about 'This food is perfect', 'Perfect people' to describe."

"POSITIVE comments talk about 'The waiter is rude', 'Their behaviour is rude', etc."

In [4]:
@labeling_function()
def keyword_less(x):
    return POSITIVE if re.search("less", x.text, flags=re.I) else ABSTAIN
@labeling_function()
def keyword_love(x):
    return NEGATIVE if re.search("love", x.text, flags=re.I) else ABSTAIN
@labeling_function()
def keyword_great(x):
    return NEGATIVE if re.search("great", x.text, flags=re.I) else ABSTAIN
@labeling_function()
def keyword_rude(x):
    return POSITIVE if re.search("rude", x.text, flags=re.I) else ABSTAIN

<div class="alert alert-success">
<b> 3.2.2 Heuristic Keywords Labeling Functions </b>

<p>When I manually labelled the development dataset at the very beginning stage,  
    I realise the POSITIVE text always come up with high useful rate and vice versa.
    Meanwhile, funny rate could 
    equal to 0, if this comment is labelled as 0. Hence, certain LFs were made.
</div>

In [5]:
@labeling_function()
def high_useful(x):
    return POSITIVE if x.useful > 10 else ABSTAIN
@labeling_function()
def low_useful(x):
    return NEGATIVE if x.useful < 2 else ABSTAIN
@labeling_function()
def funny(x):
    return NEGATIVE if x.funny == 0 else ABSTAIN

<div class="alert alert-success">
<b> 3.2.2 Summary </b>

<p>Overall, the coverage, accuracy and relationship among LFs made above can be obtained from below Label Metrics.

</div>

In [7]:
import re
lfs = [keyword_love,keyword_great,keyword_rude,keyword_less,low_useful,high_useful, funny]
applier = SparkLFApplier(lfs)
L_dev = applier.apply(df_dev.rdd)
g_label =np.array(df_dev.select('label').collect())
LFAnalysis(L_dev, lfs).lf_summary(g_label)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
keyword_love,0,[0],0.194,0.188,0.032,72,25,0.742268
keyword_great,1,[0],0.342,0.316,0.038,109,62,0.637427
keyword_rude,2,[1],0.032,0.030,0.030,14,2,0.875000
keyword_less,3,[1],0.084,0.082,0.082,29,13,0.690476
low_useful,4,[0],0.772,0.722,0.074,232,154,0.601036
high_useful,5,[1],0.010,0.008,0.008,4,1,0.800000
funny,6,[0],0.802,0.760,0.084,241,160,0.600998


In [ ]:
#Pavlos's Part:

In [ ]:
#Simon's Part:

### 3.2 Writing Labeling Functions 2: Numerical Functions

Before

In [2]:
#Jiaqi's Part

## 4. Data Augmentation

[To be continue:]

## 5. Training the Binary Classifier 

[To be continue:]

## 6. Conclusion

[To be continue:]